In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report
import joblib

In [62]:
df = pd.read_csv('../datasets/Cirrhosis.csv')
df.dropna(inplace=True)
df = df.drop(columns=['ID', 'N_Days', 'Status', 'Drug'])

In [63]:
X = df.drop(columns=['Stage'])
y = df['Stage']

categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X.select_dtypes(include=['number']).columns.tolist()
print(categorical_features)
print(numerical_features)

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])


X_processed = preprocessor.fit_transform(X)
print(f"Shape of processed data: {X_processed.shape}")


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

['Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
['Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
Shape of processed data: (276, 21)


In [64]:
# Define models
logistic_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(multi_class='ovr'))])

neural_network_model = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', MLPClassifier(max_iter=1000))])

random_forest_model = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', RandomForestClassifier())])

# Define parameter grids for hyperparameter tuning
param_grid_logistic = {
    'classifier__C': [0.1, 1, 10],
    'classifier__solver': ['liblinear', 'saga']
}

param_grid_nn = {
    'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'classifier__alpha': [0.0001, 0.001]
}

param_grid_rf = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_features': ['auto', 'sqrt'],
    'classifier__min_samples_split': [2, 5, 10]
}

In [65]:
grid_search_logistic = GridSearchCV(logistic_model, param_grid_logistic, cv=5)
grid_search_nn = GridSearchCV(neural_network_model, param_grid_nn, cv=5)
grid_search_rf = GridSearchCV(random_forest_model, param_grid_rf, cv=5)

grid_search_logistic.fit(X_train, y_train)
grid_search_nn.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)

c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppD

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Sex',
                                                                          'Ascites',
                                                                          'Hepatomegaly',
                                                                          'Spiders',
                                                                          'Edema']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Bilirubin',
                                                                          'Cholesterol',
                                                                          'Albumin',
                                                                          'Copper',
                                                                          'Alk_Phos',
                                                                          'SGOT',
                                                                          'Tryglicerides',
                                                                          'Platelets',
                                                                          'Prothrombin'])])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_features': ['auto', 'sqrt'],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 200]})

In [66]:
# Ensemble methods
voting_clf = VotingClassifier(estimators=[
    ('logistic', grid_search_logistic.best_estimator_),
    ('nn', grid_search_nn.best_estimator_),
    ('rf', grid_search_rf.best_estimator_)
], voting='soft')

voting_clf.fit(X_train, y_train)

c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


VotingClassifier(estimators=[('logistic',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('cat',
                                                                                OneHotEncoder(handle_unknown='ignore'),
                                                                                ['Sex',
                                                                                 'Ascites',
                                                                                 'Hepatomegaly',
                                                                                 'Spiders',
                                                                                 'Edema']),
                                                                               ('num',
                                                                                StandardScaler(),
                                                                                ['Age',
                                                                                 'Bilirubin',
                                                                                 'Cholesterol',
                                                                                 'Albumin',
                                                                                 'Copper',
                                                                                 'Alk_Phos',
                                                                                 'SGOT',
                                                                                 'Tryglicerides',
                                                                                 'Platelets',
                                                                                 'Prothrombin'])])),
                                              ('clas...
                                               ColumnTransformer(transformers=[('cat',
                                                                                OneHotEncoder(handle_unknown='ignore'),
                                                                                ['Sex',
                                                                                 'Ascites',
                                                                                 'Hepatomegaly',
                                                                                 'Spiders',
                                                                                 'Edema']),
                                                                               ('num',
                                                                                StandardScaler(),
                                                                                ['Age',
                                                                                 'Bilirubin',
                                                                                 'Cholesterol',
                                                                                 'Albumin',
                                                                                 'Copper',
                                                                                 'Alk_Phos',
                                                                                 'SGOT',
                                                                                 'Tryglicerides',
                                                                                 'Platelets',
                                                                                 'Prothrombin'])])),
                                              ('classifier',
                                               RandomForestClassifier(min_samples_split=10,
                                                                      n_estimators=50))]))],
               

In [67]:
# Evaluate ensemble model
y_pred_voting = voting_clf.predict(X_test)
print("Ensemble Voting Classifier:")
print(classification_report(y_test, y_pred_voting))

# Save the final model to disk
joblib.dump(preprocessor, 'cirrhosis_preprocessor.pkl')
joblib.dump(voting_clf, 'cirrhosis_model.pkl')
print("Model saved")

Ensemble Voting Classifier:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         1
         2.0       0.16      0.27      0.20        11
         3.0       0.44      0.29      0.35        24
         4.0       0.67      0.70      0.68        20

    accuracy                           0.43        56
   macro avg       0.32      0.32      0.31        56
weighted avg       0.46      0.43      0.43        56

Model saved


c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo